In [1]:
# IPython powertools
from IPython.display import clear_output, HTML, display, Javascript

# The project formally known as IPython.widgets
import ipywidgets as widgets

# Will need this for our backend
from sqlalchemy import table, MetaData, select

# Our old friends
import pandas as pd
import numpy as np

import qgrid

import pyfolio

cells_visable = True
def toggle_code_cells(btn):
    global cells_visable
    if cells_visable:
        display(Javascript("$('div.input').hide();"))
        btn.description = "Show Code Cells"
    else:
        display(Javascript("$('div.input').show();"))
        btn.description = "Hide Code Cells"
    cells_visable = not cells_visable
    

toggle_btn = widgets.Button(description="Hide Code Cells")
toggle_btn.on_click(toggle_code_cells)

display(toggle_btn)



In [41]:
css ="""<style> .container { width:60% !important; } </style>"""
display(HTML(css))

# Bootstrapping Internal Applications With IPython Widgets
<br> </br>
###  Andrew Campbell 
<img src="files/QuantopianLogo.png"  width="400">


## A crowd sourced hedge fund?


- Free set of tools for researching, building, and deploying stock trading algorithms.

- 53,000 users.

- 700,000 code snapshots in our point-in-time database.

- We only see data exaust, not the user's code.

- Finding and investing in the top .01% of algorithmic investing talent.

- Algorithm authors keep a cut of the profit they generate.


## From 700,000 to 10

<center><img src='files/pipeline3.png' align='middle'></center>

# Sometimes data science is manual

### Pyfolio https://www.github.com/quantopian/pyfolio
<div>
<img src='all_tear_2.png'>
</div>

# Bookkeeping

## The Google doc days...
<img src="files/google_doc3.png" width=2000>

doesn't scale

we were making this up as we went along. Can't go buy a book or look up a instructables article on how to crowdsource a hedge fund.

image of instructables how to crowdsource a hedge fund image

## JupyterHub data science stack

Multi-user Jupyter (IPython) notebook server.

Leverage computing power of AWS.

Easily share research and tools.

Docker:
https://hub.docker.com/r/twiecki/pydata-docker-jupyterhub/

## Started with simple functions

In [2]:
from sqlalchemy import Table, MetaData, create_engine

def write_grade_to_db(algo_id, grader, grade=None, comment=None):
    engine = create_engine(conn_str, echo=False)
    metadata = MetaData()
    metadata.reflect(bind=engine, schema='fund_stats')
    grades_t = Table('grades', MetaData(),
                     Column('algo_id', Integer, primary_key=True),
                     Column('grader', String),
                     Column('grade', String),
                     Column('comment', String),
                     schema='fund_stats')
    
    engine.execute(grades_t.insert().values(algo_id=algo_id,
                                            grader=grader,
                                            grade=grade,
                                            comment=comment))
    
    
    


## Not quite there yet...

Grader name entered wrong!

Use of non-uniform set of grades!

Undocumented functions scattered around a notebook! 

We need something productized.

We need **FAST** itteration.



build things for people who arent familar with what you are making. should be usable right away... Hard to explain functions. 
Don't want to devote the dev time to me stumbling through deploying a flask app

<center><h1>Widgets!</h1></center>


## What are widgets?

Interactive arm of the Jupyter project: https://github.com/ipython/ipywidgets/

So far largely focused on digesting data.

Why not jump off our JupyterHub platform and make a full blown app?



## How do they work?

- Comms API syncronizes Python model with Javascript view.
- Handles websocket complexity for us.
- Asychronous awesomeness taken care of. 




# Some basics

## forms

In [3]:
from ipywidgets import widgets
from IPython.display import display

grade_select = widgets.Dropdown(options=["A", "A-", "B+", 
                                         "B", "C", "D", "zz"])
grade = grade_select.selected_label
algo_id_field = widgets.IntText()
algo_id = algo_id_field.value
comment_field = widgets.Textarea(value='Enter Comment Here')

# display() renders the widget view 
display(algo_id_field, grade_select, comment_field)
print(grade_select.selected_label)
print(algo_id_field.value)

A
0


get_state method can expose all the attributes of a widget

In [4]:
def write_grade_to_db(algo_id, grader, grade=None, comment=None):
    return "OK"

## Buttons

In [6]:
def grade(btn):
    write_grade_to_db(algo_id_field.value, 'master', 
                      grade=grade_select.selected_label, 
                      comment=comment_field.value)
    print "Grade Submitted!"


grade_btn_1 = widgets.Button(description="Grade")
grade_btn_1.button_style = ''
grade_btn_1.on_click(grade)

display(grade_btn_1)

Buttons are stateless, so we need to register an action to make them send comms to our backend.
register a callback much the way you would register a callback in Javascript.

In [7]:
grade_btn = widgets.Button(description="Grade")
tearsheet_btn = widgets.Button(description='Run Tearsheet')
tearsheet_btn.margin = 5
grade_select.margin = 5
grade_btn.margin = 5
comment_field.margin = 5
algo_id_field.height = 45
grade_select.height = 45
comment_field.width = 378
grade_btn.button_style = 'info'

## styling groups of widgets

In [8]:
grade_top_row = widgets.HBox()

hid_txt = widgets.HTML("Algo ID:")
hid_txt.font = 'helvetica'
hid_txt.font_size = 20
hid_txt.margin = 5

grade_top_row.children = (hid_txt, algo_id_field, grade_select)
grade_top_row.align = 'center'
grade_bottom_row = widgets.HBox()
grade_bottom_row.children = (grade_btn, tearsheet_btn)

grade_form = widgets.VBox()
grade_form.children = (grade_top_row, comment_field, grade_bottom_row)

display(grade_form)

### QGrid
https://github.com/quantopian/qgrid

In [9]:
import pandas as pd
import qgrid
qgrid.nbinstall()

In [10]:
import pandas as pd
import qgrid
qgrid.nbinstall()
algo_grade_data = pd.read_csv('example_grading_queue.csv', 
                              parse_dates=['created_at', 'backtest_start', 
                                           'backtest_end'])
grid = qgrid.show_grid(algo_grade_data)
display(grid)

In [11]:
q_grading = widgets.VBox()
q_grading.children = (grid, grade_form)
display(q_grading)

# Onwards!

- Pass selected row from front to back.
- Pass new grade from back to front.

## Pass selected row from view to model

In [ ]:
javascript
define(["nbextensions/widgets/widgets/js/widget", "nbextensions/widgets/widgets/js/manager"], 
function(widget, manager) {
    var QGridView_ = widget.DOMWidgetView.extend({
        render: function() {
            //...lots of important stuff...
            this.setupQGrid(dgrid, editors);
        },
        setupQGrid: function(dgrid, editors) {
            var that = this;
            this.$el.addClass('q-grid-container');
            var table = this.$el.append('div');
            table.addClass('q-grid');
            var df = JSON.parse(this.model.get('_df_json'));
            var column_types = JSON.parse(this.model.get('_column_types_json'));
            grid = new dgrid.QGrid(table[0], df, column_types);
                
            //...lots of other important stuff...
        
            sgrid.onMouseDown.subscribe(function(e, args) {
                var row_data = grid.slick_grid.row_data[args.row];
                var msg = {'data': row_data, 'type': 'selected_row_change'};
                that.send(msg);
            });
        }
    });
});

## Make the relevant row data available as a traitlet

In [12]:
from traitlets import Unicode, Instance, Integer

class QGridWidget(widgets.DOMWidget):
    _view_module = Unicode("nbextensions/qgridjs/qgrid.widget", sync=True)
    _view_name = Unicode('QGridView', sync=True)
    
    # add a traitlet 
    selected_id = Integer(0, sync=True)
               
    self.on_msg(self._handle_qgrid_msg)

    def _handle_qgrid_msg(self, widget, content, buffers=None):
        if content['type'] == 'selected_row_change':
            row_data = content['data']
            self.selected_id = row_data['algo_id']

In [13]:
from traitlets import link

link((algo_id_field, 'value'), (grid, 'selected_id'))
display(q_grading)

## Passing new grades from model to view

In [14]:
class QGridWidget(widgets.DOMWidget):
    _view_module = Unicode("nbextensions/qgridjs/qgrid.widget", sync=True)
    _view_name = Unicode('QGridView', sync=True)
    
    # add a traitlet
    selected_id = Integer(0, sync=True)

    def update_grade(self, algo_id, grader, grade):
        row_ix = self.df[self.df.algo_id == algo_id].index.values[0]
        col_ix = self.df.columns.get_loc(grader)
        msg = {
            'type': 'grade_cell_updated',
            'row_ix': row_ix,
            'col_ix': col_ix,
            'new_val': grade,
        }
        self.send(msg)


In [ ]:
javascript
define(["nbextensions/widgets/widgets/js/widget", "nbextensions/widgets/widgets/js/manager"], 
function(widget, manager) {
    var QGridView_ = widget.DOMWidgetView.extend({
        render: function() {
            //...lots of important stuff...
            this.setupQGrid(dgrid, editors);
        },
        setupQGrid: function(dgrid, editors) {                
            //...lots of other important stuff...
            this.model.on('msg:custom', this.handleMsg, this);
        },
        
        handleMsg: function(msg) {
            var sgrid = grid.slick_grid; 
            if (msg.type == 'grade_cell_updated'){
                var cell = sgrid.getCellNode(msg.row_ix, msg.col_ix)
                cell.innerText = msg.new_val
            }
        }
    });    
});

## Know your user

In [14]:
def get_user():
    usernames = {'acampbell': 'Andrew',
                 'wiecki': 'Thomas',
                 'ddunn': 'Dan',
                 'fawce': 'fawce',
                 'jlent': 'Justin',
                 'jstauth': 'Jess'}
    resp = !echo "$USER"
    bash_usr = resp[0]
    user = usernames[bash_usr]
    return user

def v_grade(btn):
    _grade = grade_select.selected_label
    aid = algo_id_field.value
    usr = get_user()
    resp = write_grade_to_db(aid, usr, grade=_grade, 
                             comment=comment_field.value)
    if resp == 'OK':
        grid.update_grade(aid, usr + "_grade", _grade)

q_grading.children[1].children[-1].children[0].on_click(v_grade)

## Now make it look professional!

In [15]:
cells_visable = True
def toggle_code_cells(btn):
    global cells_visable
    if cells_visable:
        display(Javascript("$('div.input').hide();"))
        btn.description = "Show Code Cells"
    else:
        display(Javascript("$('div.input').show();"))
        btn.description = "Hide Code Cells"
    cells_visable = not cells_visable
    

toggle_btn = widgets.Button(description="Hide Code Cells")
toggle_btn.on_click(toggle_code_cells)

display(toggle_btn)

## Another display gem

In [16]:
css ="""<style> .container { width:90% !important; } </style>"""
display(HTML(css))

In [17]:
display(toggle_btn, q_grading)